Build P3HT bulk simulation 

In [1]:
import os,sys, copy, pickle

In [2]:
from streamm import *

In [3]:
%load_ext autoreload
%autoreload 2

Load resources from previous example 

In [4]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_local = resource.Resource(res_tag)
res_local.load_json()

Reading in resource json file res_local.json 


No handlers could be found for logger "streamm.resource"


In [5]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


In [6]:
print os.getcwd()

/Users/tkemper/Development/streamm-tools/examples


Load oligomer from previous examples 

In [8]:
p3ht_n10_esp = buildingblock.Container('p3ht_n10_esp')

In [9]:
p3ht_n10_esp.read_cply()

IOError: [Errno 2] No such file or directory: 'p3ht_n10_esp.cply'

In [ ]:
p3ht_n10_esp.lat_cubic(50.0)

In [ ]:
p3ht_n10_esp.guess_oplsa()

In [ ]:
p3ht_n10_esp.bonded_bonds()
p3ht_n10_esp.bonded_angles()
p3ht_n10_esp.bonded_dih()

Place the oligomer into a simulation and add parameters 

In [ ]:
lmp_oligo = lammps.LAMMPS('p3ht_n10_esp_nvt')
lmp_oligo.load_json()

In [ ]:
lmp_oligo.set_resource(peregrine)

In [ ]:
lmp_oligo.properties['scratch'] = lmp_oligo.dir['scratch']

In [ ]:
lmp_oligo.make_dir()

In [ ]:
lmp_oligo.add_strucC(p3ht_n10_esp)

In [ ]:
print lmp_oligo.strucC.lat._matrix

In [ ]:
lmp_oligo.read_param("conj.param")

In [ ]:
lmp_oligo.set_ffparam()

In [ ]:
for p,pt in lmp_oligo.strucC.particles.iteritems(): # properties["charge"] 
    print p,pt.properties["symbol"],pt.properties["fftype"]

Read in templates

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_nvt.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_oligo.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_oligo.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_oligo.dir['launch']

In [ ]:
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.load_str('templates','run')

In [ ]:
lmp_oligo.load_str('templates','in')

In [ ]:
lmp_oligo.write_data()

In [ ]:
lmp_oligo.replacewrite_prop('in','input','in','%s.in'%(lmp_oligo.tag))

In [ ]:
lmp_oligo.properties['input_in'] = lmp_oligo.files['input']['in']

In [ ]:
lmp_oligo.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_oligo.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_oligo.tag)
lmp_oligo.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_oligo.dir['home'])
lmp_oligo.dump_json()
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.push()

In [ ]:
lmp_oligo.run()

In [ ]:
lmp_oligo.check()
print lmp_oligo.meta['status']

In [ ]:
lmp_oligo.analysis()
print lmp_oligo.meta['status']

In [ ]:
print lmp_oligo.files['output']

In [ ]:
os.chdir(lmp_oligo.dir['home'])
lmp_oligo.dump_json()
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.meta['status'] = 'finished'

In [ ]:
lmp_oligo.store()

In [ ]:
print lmp_oligo.files['output']

In [ ]:
lmp_oligo.pull()

In [ ]:
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.read_data(lmp_oligo.files['output']['data_3'])

In [ ]:
lmp_oligo.strucC.write_cply()

In [ ]:
lmp_oligo.strucC.group_prop('mol','mol')

In [ ]:
lmp_oligo.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_oligo.strucC.tag 

In [ ]:
lmp_oligo.strucC.tag = "%s_nvt"%(lmp_oligo.strucC.tag )

In [ ]:
os.chdir(lmp_oligo.dir['home'])

In [ ]:
lmp_oligo.dump_json()

In [ ]:
lmp_oligo.strucC.write_xyz()

In [ ]:
lmp_oligo.strucC.write_cply()

In [ ]:
p3ht_n10_nvt = copy.deepcopy(lmp_oligo.strucC)

Replicate the oligomer into large simulation cell

In [1]:
xO = 20

In [ ]:
p3ht_n10_esp_x = buildingblock.Container('p3ht_n10_x%d'%(xO))
p3ht_n10_esp_x.read_cply()

In [ ]:
p3ht_n10_esp_x.lat_cubic(150.0)

In [ ]:
print p3ht_n10_esp_x.lat._matrix

Set seed 

In [ ]:
s = 2984927

In [ ]:
p3ht_n10_esp_x = p3ht_n10_esp_x.add_struc(p3ht_n10_nvt,xO,s)

In [ ]:
p3ht_n10_esp_x.tag = 'p3ht_n10_x%d'%(xO)

In [ ]:
p3ht_n10_esp_x.read_cply()

In [ ]:
p3ht_n10_esp_x.bonded_bonds()
p3ht_n10_esp_x.bonded_angles()
p3ht_n10_esp_x.bonded_dih()

In [ ]:
p3ht_n10_esp_x.write_xyz()

In [ ]:
p3ht_n10_esp_x.write_cply()

In [ ]:
print p3ht_n10_esp_x.n_bonds,p3ht_n10_esp_x.n_angles,p3ht_n10_esp_x.n_dihedrals

Create LAMMPS calculation to equilibrate bulk system

In [ ]:
xO = 20

In [ ]:
lmp_bulk = lammps.LAMMPS('p3ht_n10_esp_x%d'%(xO))

In [ ]:
lmp_bulk.load_json()

In [ ]:
lmp_bulk.set_resource(peregrine)

In [ ]:
lmp_bulk.properties['scratch'] = lmp_bulk.dir['scratch']

In [ ]:
print lmp_bulk.tag

In [ ]:
lmp_bulk.make_dir()

In [ ]:
lmp_bulk.strucC = buildingblock.Container()

In [ ]:
lmp_bulk.add_strucC(p3ht_n10_esp_x)

In [ ]:
print lmp_bulk.strucC.n_particles

In [ ]:
lmp_bulk.read_param("conj.param")

In [ ]:
lmp_bulk.set_ffparam()

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_nvt.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_bulk.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_bulk.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_bulk.dir['launch']

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.load_str('templates','run')

In [ ]:
lmp_bulk.load_str('templates','in')

In [ ]:
lmp_bulk.write_data()

In [ ]:
lmp_bulk.replacewrite_prop('in','input','in','%s.in'%(lmp_bulk.tag))

In [ ]:
lmp_bulk.properties['input_in'] = lmp_bulk.files['input']['in']

In [ ]:
lmp_bulk.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_bulk.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_bulk.tag)
lmp_bulk.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.push()

In [ ]:
lmp_bulk.run()

In [ ]:
lmp_bulk.check()
print lmp_bulk.meta['status']

In [ ]:
lmp_bulk.analysis()
print lmp_bulk.meta['status']

In [ ]:
print lmp_bulk.files['output']

In [ ]:
print lmp_bulk.dir

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.meta['status'] = 'finished'

In [ ]:
lmp_bulk.store()

In [ ]:
lmp_bulk.pull()

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.read_data_pos(lmp_bulk.files['output']['data_3'])

In [ ]:
pos_nvt = copy.deepcopy(lmp_bulk.strucC.positions)

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.group_prop('mol','mol')

In [ ]:
lmp_bulk.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_bulk.tag 

In [ ]:
lmp_bulk.strucC.tag = "%s_nvt"%(lmp_bulk.strucC.tag )

Write new data file and copy it to scratch to be stored with output, 
since LAMMPS does not print dihedral angles for opls style 

In [ ]:
data_file = '%s.data'%(lmp_bulk.tag)

In [ ]:
lmp_bulk.write_data(data_file)

In [ ]:
lmp_bulk.add_file('output','data_4',data_file)

In [ ]:
ssh_id = "%s@%s"%(lmp_bulk.resource.ssh['username'],lmp_bulk.resource.ssh['address'])
from_pathfile = os.path.join(lmp_bulk.dir['launch'],data_file)
bash_command = "scp %s %s:%s"%(from_pathfile,ssh_id,lmp_bulk.dir['scratch'])
print bash_command
os.system(bash_command)

In [ ]:
lmp_bulk.meta['status'] = 'finished'
print lmp_bulk.meta['status']

In [ ]:
lmp_bulk.store()

In [ ]:
os.chdir(lmp_bulk.dir['home'])

In [ ]:
lmp_bulk.dump_json()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.write_data()

In [ ]:
print lmp_bulk.tag

Run Equilibration 

In [ ]:
lmp_equ = lammps.LAMMPS('p3ht_n10_x20_equ')
lmp_equ.load_json()

In [ ]:
lmp_equ.set_resource(peregrine)

In [ ]:
lmp_equ.properties['scratch'] = lmp_equ.dir['scratch']

In [ ]:
lmp_equ.make_dir()

In [ ]:
lmp_equ.add_refcalc(lmp_bulk)

In [ ]:
lmp_equ.properties['nodes'] = 4
lmp_equ.properties['nproc'] = lmp_equ.properties['nodes']*lmp_equ.properties['ppn']
print lmp_equ.properties

In [ ]:
lmp_equ.files['input']['data_file'] = lmp_bulk.files['output']['data_4']
print lmp_equ.files['input']['data_file']

In [ ]:
lmp_equ.properties['data_file']  = lmp_equ.files['input']['data_file'] 

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_equ1.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_equ.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_equ.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_equ.dir['launch']

In [ ]:
os.chdir(lmp_equ.dir['launch'])

In [ ]:
lmp_equ.load_str('templates','run')

In [ ]:
lmp_equ.load_str('templates','in')

In [ ]:
lmp_equ.replacewrite_prop('in','input','in','%s.in'%(lmp_equ.tag))

In [ ]:
lmp_equ.properties['input_in'] = lmp_equ.files['input']['in']

In [ ]:
lmp_equ.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_equ.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_equ.tag)
lmp_equ.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_equ.dir['home'])
lmp_equ.dump_json()
os.chdir(lmp_equ.dir['launch'])

In [ ]:
ref_calc = lmp_equ.references['p3ht_n10_esp_x20']

In [ ]:
print ref_calc.files

In [ ]:
lmp_equ.push()

In [ ]:
lmp_equ.run()

In [ ]:
lmp_equ.check()
print lmp_equ.meta['status']

In [ ]:
lmp_bulk.analysis()
print lmp_bulk.meta['status']

In [ ]:
print lmp_bulk.files['output']

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.meta['status'] = 'finished'

In [ ]:
lmp_bulk.store()

In [ ]:
lmp_bulk.pull()

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.read_data(lmp_bulk.files['output']['data_3'])

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.group_prop('mol','mol')

In [ ]:
lmp_bulk.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_bulk.strucC.tag 

In [ ]:
lmp_bulk.strucC.tag = "%s_nvt"%(lmp_bulk.strucC.tag )

In [ ]:
os.chdir(lmp_bulk.dir['home'])

In [ ]:
lmp_equ.dump_json()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.write_cply()

Analyize bulk properties with streamm

In [8]:
lmp_nvt = lammps.LAMMPS('p3ht_n10_x20_nvt')

In [9]:
p3ht_n10_x20_nvt = buildingblock.Container('p3ht_n10_x20_nvt')
p3ht_n10_x20_nvt.read_cply()

In [10]:
lmp_nvt.set_strucC(p3ht_n10_x20_nvt)

In [11]:
lmp_nvt.read_data_pos('npt5.data')

In [12]:
lmp_nvt.strucC.tag = 'p3ht_n10_x20_npt5'

In [13]:
lmp_nvt.strucC.write_xyz()

In [14]:
lmp_nvt.strucC.write_cply()

In [ ]:
lmp_nvt.strucC.